# 1. 프로젝트 실습

### 데이터 불러오기

[데이터타입 통합]
kopo_channel_seasonality_new.csv 파일을 불러온 후
selloutData 변수에 담으세요 이후 
QTY컬럼→실수(float), 이외컬럼→문자(str)로 변경하세요

In [77]:
import pandas as pd

selloutData = \
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

selloutData = selloutData.astype({"REGIONID":str,
                    "PRODUCT":str,
                    "YEARWEEK":str,
                    "QTY":float})

### 1. 데이터 전처리

### A-1. 불량데이터 처리

[불량 데이터 처리]
kopo_channel_seasonality_new.csv 자료를 담은 
selloutData 변수에서
QTY컬럼 음수(반품)인 경우 0, 양수인 경우 기존 QTY 값유지하는 로직을 적용하여 QTY_NEW 컬럼을 추가하세요




In [78]:
### 함수정의: 판매량 데이터를 정제 (-값: 0, 이외 기존값)
### 파라미터 inQty: 실제 판매량
def cleanQty(inQty):

    # 테스트데이터: inQty
    # inQty = 500
    newQty = 0

    if inQty <= 0:
        newQty = 0
    else:
        newQty = inQty

    return newQty

selloutData["NEW_QTY"] = \
    selloutData.QTY.apply(cleanQty)

### A-2. 데이터 통합

[데이터 통합]
selloutData 자료에서 
YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인
데이터만 조회한 후 refinedSelloutData 변수에 담으세요 (YEAR, WEEK는 모두 str)


In [79]:
selloutData["YEAR"] = selloutData.YEARWEEK.str[0:4]
selloutData["WEEK"] = selloutData.YEARWEEK.str[4:6]

refinedSelloutData = selloutData.copy()

In [80]:
refinedSelloutData.shape

(124658, 7)

### B-3. 추세선 도출

0.[데이터 통합]
refinedSelloutData 자료에서 
YEAR, WEEK 컬럼을 생성하고 WEEK 가 52 이하인
데이터만 조회한 후 refinedSelloutData 변수에 담으세요 -> 53주차를 제거한다

In [82]:
refinedSelloutData.shape

(124658, 7)

In [86]:
refinedSelloutData = \
    refinedSelloutData.loc[refinedSelloutData.WEEK != "53"]

1. refinedSelloutData 에서 →
지역, 상품, 연주차 컬럼순으로 오름차순 정렬하여
sortedData 변수에 담으세요

In [74]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK"]

In [89]:
sortedData = refinedSelloutData.sort_values(by=sortKey)

2. [지역, 상품, 연도 별 집계] * 그룹키 = 지역(REGIONID),상품(PRODUCT) 연도(YEAR)
groupKey = ["REGIONID","PRODUCT","YEAR"]
sortedData 에서 지역, 상품, 연도 단위
판매량(QTY_NEW) 의 평균 연산 후
groupData 변수에 담으세요 이후 컬럼명을 QTY_MEAN로 변경하세요


In [93]:
groupKey = ["REGIONID","PRODUCT","YEAR"]

In [96]:
groupData= refinedSelloutData.groupby(by=groupKey)["NEW_QTY"].agg(["mean"]).reset_index()

In [99]:
groupData.rename(columns={"mean":"QTY_MEAN"},inplace=True)

In [118]:
# 선택한 조건에 맞는 데이터 추출
targetRegion = "A00"
targetProduct = "PRODUCT34"
targetYear = "2014"

In [117]:
groupData.loc[(groupData.REGIONID==targetRegion) &
              (groupData.PRODUCT==targetProduct) &
              (groupData.YEAR==targetYear)].QTY_MEAN.values[0]

275.96153846153845